In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange

In [2]:
import torch
import torch.nn as nn
import pandas as pd 

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

data_type=torch.float32

In [3]:
x0 = 100.0 # initial condition
sigma = 0.15 # volatility
d=5
K=100.0

segs=20
r = 0.05 # risk free rate
batch_size = 100 # batch size
steps=1000
T = 1 # maturity
dt = T/steps # mesh size

sqrt_dt=np.sqrt(dt); 
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level logsig 91 sig 258

In [4]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [5]:
#dW=np.sqrt(dt)*np.random.normal(size=(n_path, steps,d))

In [6]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,5))*100.0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*s_vec[-1]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec

def ComputeMultiLevelSig(path, number_of_segment, depth,log_sig=False):
    n_batch, nsteps, chanels = path.shape
    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []
    s=iisig.prepare(d+1,depth)
    
    if log_sig: 
        ll=iisig.logsig(np.expand_dims(path[:,0,:],axis=1),s)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.logsig(path[:,0:t_vec[i+1]+1,:],s)) ##if not
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    else: 
        ll=iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.sig(path[:,0:t_vec[i+1]+1,:],depth)) ##if not
            #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
        
    return MultiLevelSig

In [7]:
### The x_ten here has to be a path
### The level of discretization is on the fine mesh
def terminal_loss(x_ten):
    res=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    return res

In [8]:
def generate_samples(batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    sigs=ComputeMultiLevelSig(S_timePath, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [9]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    dim=258; 
    Ntime=20; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,d] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [10]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.tanh(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE_Backward(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE_Backward,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        
    def forward(self,batch_sig,batch_dW,batch_YT, batch_x, batch_sel): 
        Y=batch_YT
        for i in np.arange(segs-1,-1, -1):
            Y=Y-Y*r*T/segs-sigma*torch.sum( self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
            gi=terminal_loss(batch_x[:,:batch_sel[i],:])
            Y=torch.relu(gi-Y)+Y
        return Y

In [11]:
def loss_var(x):
    temp=torch.var(x)
    return temp

In [31]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE_bw=Lookback_PPDE_Backward(cfg)
model_PPDE_bw
optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
grad_clip=0.1
y0_mean=[];
loss_vec=[];
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

## 0.5823 -- 0.5784 -- 0.5862

In [32]:
import time

In [33]:
start_time = time.time()
for i in range(100):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=500)

    x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
    loss_temp=loss_var(x_temp)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
#   scheduler.step()
    y0_val=x_temp.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)
    
    print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)
print("--- %s seconds ---" % (time.time() - start_time))

Iter: 0 The mean Y0 is 3.0439456 Variance is: 8.95848
Iter: 1 The mean Y0 is 3.2610576 Variance is: 10.11592
Iter: 2 The mean Y0 is 3.106754 Variance is: 9.597261
Iter: 3 The mean Y0 is 3.05299 Variance is: 9.049848
Iter: 4 The mean Y0 is 2.9672103 Variance is: 9.114802
Iter: 5 The mean Y0 is 3.057727 Variance is: 9.53749
Iter: 6 The mean Y0 is 3.3406546 Variance is: 8.946721
Iter: 7 The mean Y0 is 3.0261736 Variance is: 8.346736
Iter: 8 The mean Y0 is 3.1942303 Variance is: 9.195718
Iter: 9 The mean Y0 is 2.975846 Variance is: 9.030522
Iter: 10 The mean Y0 is 3.1405969 Variance is: 8.482119
Iter: 11 The mean Y0 is 3.1686642 Variance is: 9.387202
Iter: 12 The mean Y0 is 3.0223837 Variance is: 8.9903
Iter: 13 The mean Y0 is 2.8949578 Variance is: 8.5382805
Iter: 14 The mean Y0 is 3.1684904 Variance is: 10.223728
Iter: 15 The mean Y0 is 3.1074357 Variance is: 9.315168
Iter: 16 The mean Y0 is 3.1677694 Variance is: 11.1267395
Iter: 17 The mean Y0 is 2.957718 Variance is: 8.749437
Iter: 18

In [99]:
y_pred=np.array(y0_mean)
loss_var=np.array(loss_vec)
iters=np.arange(0,len(y_pred),1)+1

In [100]:
df=pd.DataFrame()
df['iter']=iters
df['y_pred']=y_pred
df['loss_var']=loss_var

In [101]:
#df.to_csv('eg2_trained_data/method2_2.csv')

In [102]:
start_idx=50
df[start_idx:][df[start_idx:].loss_var<9.5].y_pred.mean()

3.102689743041992

In [103]:
df[start_idx:][df[start_idx:].loss_var<9.5].shape

(61, 3)

In [22]:
for epoch in np.arange(10,51,1):
    model_PPDE_bw=Lookback_PPDE_Backward(cfg)
    optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
    grad_clip=0.2
    y0_mean=[];
    loss_vec=[];
    for i in range(100):
        batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=500)

        x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
        loss_temp=loss_var(x_temp)

        optimizer.zero_grad()
        loss_temp.backward()
        optimizer.step()
    #   scheduler.step()
        y0_val=x_temp.mean().cpu().detach().numpy()
        loss_val=loss_temp.cpu().detach().numpy()

        y0_mean.append(y0_val)
        loss_vec.append(loss_val)
        if grad_clip: 
            nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)

        print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)
    
    print("Epoch:", epoch)
    
    y_pred=np.array(y0_mean)
    loss_val=np.array(loss_vec)
    iters=np.arange(0,len(y_pred),1)+1
    df=pd.DataFrame()
    df['iter']=iters
    df['y_pred']=y_pred
    df['loss_var']=loss_val
    df.to_csv('eg2_trained_data/method2'+'_'+str(epoch)+'.csv')

Iter: 0 The mean Y0 is 3.0636036 Variance is: 8.918401
Iter: 1 The mean Y0 is 3.1418602 Variance is: 9.556008
Iter: 2 The mean Y0 is 3.1843872 Variance is: 9.552142
Iter: 3 The mean Y0 is 3.124587 Variance is: 9.435904
Iter: 4 The mean Y0 is 3.063492 Variance is: 9.218032
Iter: 5 The mean Y0 is 2.9838872 Variance is: 8.619873
Iter: 6 The mean Y0 is 3.070615 Variance is: 8.226476
Iter: 7 The mean Y0 is 3.1675434 Variance is: 9.423783
Iter: 8 The mean Y0 is 2.8982742 Variance is: 9.307381
Iter: 9 The mean Y0 is 3.0833254 Variance is: 9.617275
Iter: 10 The mean Y0 is 2.81708 Variance is: 8.088984
Iter: 11 The mean Y0 is 2.945124 Variance is: 8.835895
Iter: 12 The mean Y0 is 3.0235858 Variance is: 9.053851
Iter: 13 The mean Y0 is 3.204056 Variance is: 9.614461
Iter: 14 The mean Y0 is 2.807321 Variance is: 7.6606946
Iter: 15 The mean Y0 is 3.152049 Variance is: 8.655065
Iter: 16 The mean Y0 is 3.1752617 Variance is: 9.146013
Iter: 17 The mean Y0 is 3.158823 Variance is: 9.611385
Iter: 18 Th

Iter: 48 The mean Y0 is 3.057063 Variance is: 8.6131115
Iter: 49 The mean Y0 is 2.9546137 Variance is: 8.638674
Iter: 50 The mean Y0 is 3.026467 Variance is: 9.403893
Iter: 51 The mean Y0 is 3.3270464 Variance is: 10.335802
Iter: 52 The mean Y0 is 3.2110763 Variance is: 9.208907
Iter: 53 The mean Y0 is 3.0809932 Variance is: 8.79777
Iter: 54 The mean Y0 is 3.0875409 Variance is: 8.719593
Iter: 55 The mean Y0 is 3.138045 Variance is: 8.867907
Iter: 56 The mean Y0 is 2.918024 Variance is: 9.109741
Iter: 57 The mean Y0 is 3.034061 Variance is: 8.695923
Iter: 58 The mean Y0 is 3.4416106 Variance is: 10.193864
Iter: 59 The mean Y0 is 3.0623598 Variance is: 8.816448
Iter: 60 The mean Y0 is 2.887903 Variance is: 8.247346
Iter: 61 The mean Y0 is 2.9800434 Variance is: 9.544861
Iter: 62 The mean Y0 is 2.999139 Variance is: 8.276843
Iter: 63 The mean Y0 is 3.055503 Variance is: 8.773513
Iter: 64 The mean Y0 is 3.3802214 Variance is: 10.355365
Iter: 65 The mean Y0 is 3.09243 Variance is: 8.47273


Iter: 95 The mean Y0 is 3.3395019 Variance is: 9.291488
Iter: 96 The mean Y0 is 3.276533 Variance is: 9.336229
Iter: 97 The mean Y0 is 3.2154188 Variance is: 8.948151
Iter: 98 The mean Y0 is 3.1846933 Variance is: 8.29661
Iter: 99 The mean Y0 is 3.129874 Variance is: 8.494512
Epoch: 12
Iter: 0 The mean Y0 is 3.102556 Variance is: 9.772108
Iter: 1 The mean Y0 is 3.44714 Variance is: 10.666774
Iter: 2 The mean Y0 is 3.1698976 Variance is: 9.001837
Iter: 3 The mean Y0 is 3.1828675 Variance is: 9.46211
Iter: 4 The mean Y0 is 3.0440638 Variance is: 8.5717945
Iter: 5 The mean Y0 is 3.1025844 Variance is: 8.929769
Iter: 6 The mean Y0 is 3.0612543 Variance is: 9.80469
Iter: 7 The mean Y0 is 3.2807717 Variance is: 9.868154
Iter: 8 The mean Y0 is 3.0335913 Variance is: 8.466493
Iter: 9 The mean Y0 is 3.2820537 Variance is: 9.717329
Iter: 10 The mean Y0 is 3.1269798 Variance is: 9.074561
Iter: 11 The mean Y0 is 3.2930725 Variance is: 9.125716
Iter: 12 The mean Y0 is 3.3107905 Variance is: 9.79657

Iter: 42 The mean Y0 is 3.363346 Variance is: 9.561989
Iter: 43 The mean Y0 is 3.159472 Variance is: 9.648572
Iter: 44 The mean Y0 is 3.0088317 Variance is: 8.590101
Iter: 45 The mean Y0 is 3.19426 Variance is: 9.613708
Iter: 46 The mean Y0 is 3.0799737 Variance is: 9.574024
Iter: 47 The mean Y0 is 3.282091 Variance is: 9.81689
Iter: 48 The mean Y0 is 2.9970777 Variance is: 8.2990675
Iter: 49 The mean Y0 is 3.1333973 Variance is: 10.798645
Iter: 50 The mean Y0 is 3.0075762 Variance is: 8.389718
Iter: 51 The mean Y0 is 3.1921458 Variance is: 10.002568
Iter: 52 The mean Y0 is 3.2192814 Variance is: 9.507541
Iter: 53 The mean Y0 is 3.437665 Variance is: 10.679967
Iter: 54 The mean Y0 is 3.0937624 Variance is: 8.588572
Iter: 55 The mean Y0 is 3.1247778 Variance is: 10.3052225
Iter: 56 The mean Y0 is 3.1785328 Variance is: 9.180662
Iter: 57 The mean Y0 is 3.2206326 Variance is: 9.347713
Iter: 58 The mean Y0 is 2.9371636 Variance is: 8.454314
Iter: 59 The mean Y0 is 3.19024 Variance is: 10.0

Iter: 89 The mean Y0 is 3.1718578 Variance is: 9.405974
Iter: 90 The mean Y0 is 3.2354362 Variance is: 9.809929
Iter: 91 The mean Y0 is 3.4302435 Variance is: 9.290901
Iter: 92 The mean Y0 is 3.1448305 Variance is: 9.576908
Iter: 93 The mean Y0 is 3.367431 Variance is: 9.739934
Iter: 94 The mean Y0 is 2.9857109 Variance is: 10.151721
Iter: 95 The mean Y0 is 3.238857 Variance is: 8.892587
Iter: 96 The mean Y0 is 3.2259574 Variance is: 9.902516
Iter: 97 The mean Y0 is 3.0621974 Variance is: 9.735573
Iter: 98 The mean Y0 is 3.2426486 Variance is: 9.17444
Iter: 99 The mean Y0 is 3.165286 Variance is: 9.457788
Epoch: 15
Iter: 0 The mean Y0 is 2.9087915 Variance is: 9.403962
Iter: 1 The mean Y0 is 3.128151 Variance is: 8.850889
Iter: 2 The mean Y0 is 2.8175857 Variance is: 8.99869
Iter: 3 The mean Y0 is 3.3458943 Variance is: 9.450655
Iter: 4 The mean Y0 is 3.110091 Variance is: 9.362281
Iter: 5 The mean Y0 is 2.924554 Variance is: 8.79604
Iter: 6 The mean Y0 is 3.011959 Variance is: 9.24947

...

Iter: 89 The mean Y0 is 3.3340561 Variance is: 11.10236
Iter: 90 The mean Y0 is 2.9299524 Variance is: 8.969959
Iter: 91 The mean Y0 is 3.1956744 Variance is: 8.433856
Iter: 92 The mean Y0 is 3.1221857 Variance is: 9.812799
Iter: 93 The mean Y0 is 2.9544394 Variance is: 8.65899
Iter: 94 The mean Y0 is 3.1572146 Variance is: 9.890369
Iter: 95 The mean Y0 is 3.1452157 Variance is: 9.130611
Iter: 96 The mean Y0 is 2.9992611 Variance is: 8.88029
Iter: 97 The mean Y0 is 2.9895718 Variance is: 8.322129
Iter: 98 The mean Y0 is 2.9979405 Variance is: 8.552663
Iter: 99 The mean Y0 is 3.282799 Variance is: 9.048442
Epoch: 43
Iter: 0 The mean Y0 is 3.2431376 Variance is: 10.039462
Iter: 1 The mean Y0 is 2.9321966 Variance is: 8.863595
Iter: 2 The mean Y0 is 3.3256664 Variance is: 9.892183
Iter: 3 The mean Y0 is 2.8681705 Variance is: 8.937635
Iter: 4 The mean Y0 is 3.149661 Variance is: 9.514306
Iter: 5 The mean Y0 is 3.0237637 Variance is: 9.952293
Iter: 6 The mean Y0 is 3.2053378 Variance is: 1

Iter: 36 The mean Y0 is 2.8408484 Variance is: 8.561511
Iter: 37 The mean Y0 is 3.1388488 Variance is: 8.390147
Iter: 38 The mean Y0 is 3.079636 Variance is: 8.298249
Iter: 39 The mean Y0 is 2.9395876 Variance is: 9.187103
Iter: 40 The mean Y0 is 3.1799624 Variance is: 9.284966
Iter: 41 The mean Y0 is 3.1915326 Variance is: 8.782462
Iter: 42 The mean Y0 is 2.824499 Variance is: 8.043124
Iter: 43 The mean Y0 is 3.067812 Variance is: 8.187938
Iter: 44 The mean Y0 is 3.0081096 Variance is: 8.826229
Iter: 45 The mean Y0 is 3.1455874 Variance is: 9.634225
Iter: 46 The mean Y0 is 2.958409 Variance is: 8.10229
Iter: 47 The mean Y0 is 2.9369807 Variance is: 8.3774805
Iter: 48 The mean Y0 is 3.211481 Variance is: 10.093878
Iter: 49 The mean Y0 is 3.3940046 Variance is: 10.732939
Iter: 50 The mean Y0 is 3.2411733 Variance is: 9.712128
Iter: 51 The mean Y0 is 3.057995 Variance is: 8.955306
Iter: 52 The mean Y0 is 3.207009 Variance is: 9.850569
Iter: 53 The mean Y0 is 3.0267482 Variance is: 8.7788

Iter: 83 The mean Y0 is 3.0648692 Variance is: 8.545827
Iter: 84 The mean Y0 is 3.253562 Variance is: 10.253045
Iter: 85 The mean Y0 is 3.0803974 Variance is: 8.178012
Iter: 86 The mean Y0 is 3.2039394 Variance is: 8.972754
Iter: 87 The mean Y0 is 3.2891555 Variance is: 9.927532
Iter: 88 The mean Y0 is 3.2292242 Variance is: 8.841496
Iter: 89 The mean Y0 is 3.2183733 Variance is: 9.355224
Iter: 90 The mean Y0 is 2.9756806 Variance is: 9.555745
Iter: 91 The mean Y0 is 3.1460228 Variance is: 8.386096
Iter: 92 The mean Y0 is 3.046652 Variance is: 9.502942
Iter: 93 The mean Y0 is 3.122742 Variance is: 9.19713
Iter: 94 The mean Y0 is 3.015174 Variance is: 8.89298
Iter: 95 The mean Y0 is 2.9227598 Variance is: 8.82811
Iter: 96 The mean Y0 is 3.2598486 Variance is: 9.870192
Iter: 97 The mean Y0 is 3.188157 Variance is: 9.536571
Iter: 98 The mean Y0 is 3.2607536 Variance is: 8.966482
Iter: 99 The mean Y0 is 3.173892 Variance is: 9.841328
Epoch: 46
Iter: 0 The mean Y0 is 2.9542692 Variance is: 

Iter: 30 The mean Y0 is 3.1472664 Variance is: 9.814592
Iter: 31 The mean Y0 is 3.1329956 Variance is: 9.440022
Iter: 32 The mean Y0 is 3.01747 Variance is: 8.381011
Iter: 33 The mean Y0 is 3.3650393 Variance is: 11.001788
Iter: 34 The mean Y0 is 3.139802 Variance is: 9.414447
Iter: 35 The mean Y0 is 2.978187 Variance is: 8.908832
Iter: 36 The mean Y0 is 2.9029515 Variance is: 8.427184
Iter: 37 The mean Y0 is 3.0687888 Variance is: 8.226876
Iter: 38 The mean Y0 is 3.0161202 Variance is: 10.177425
Iter: 39 The mean Y0 is 3.2685373 Variance is: 9.458488
Iter: 40 The mean Y0 is 3.1938467 Variance is: 9.936417
Iter: 41 The mean Y0 is 3.0513656 Variance is: 8.947609
Iter: 42 The mean Y0 is 2.9600031 Variance is: 8.067838
Iter: 43 The mean Y0 is 3.198982 Variance is: 10.000648
Iter: 44 The mean Y0 is 3.3230264 Variance is: 9.576802
Iter: 45 The mean Y0 is 3.0169687 Variance is: 9.481896
Iter: 46 The mean Y0 is 3.2089136 Variance is: 9.948125
Iter: 47 The mean Y0 is 3.1259189 Variance is: 8.9

Iter: 77 The mean Y0 is 3.2050824 Variance is: 9.661364
Iter: 78 The mean Y0 is 3.262708 Variance is: 9.040567
Iter: 79 The mean Y0 is 3.1293287 Variance is: 9.137544
Iter: 80 The mean Y0 is 3.1815472 Variance is: 8.244702
Iter: 81 The mean Y0 is 3.1402993 Variance is: 8.909861
Iter: 82 The mean Y0 is 3.2534442 Variance is: 10.401182
Iter: 83 The mean Y0 is 3.089922 Variance is: 9.599421
Iter: 84 The mean Y0 is 2.9864485 Variance is: 8.285549
Iter: 85 The mean Y0 is 3.075734 Variance is: 8.87275
Iter: 86 The mean Y0 is 3.143188 Variance is: 9.236348
Iter: 87 The mean Y0 is 3.2551198 Variance is: 9.559295
Iter: 88 The mean Y0 is 2.8890023 Variance is: 8.424901
Iter: 89 The mean Y0 is 3.1876347 Variance is: 9.185198
Iter: 90 The mean Y0 is 3.127801 Variance is: 7.9167733
Iter: 91 The mean Y0 is 3.0167992 Variance is: 8.852532
Iter: 92 The mean Y0 is 3.323701 Variance is: 9.50584
Iter: 93 The mean Y0 is 3.2233298 Variance is: 9.118949
Iter: 94 The mean Y0 is 3.1879785 Variance is: 9.70819

In [23]:
import os 
files_dir=os.getcwd()+'\\eg2_trained_data\\'
file_names=os.listdir(files_dir)
pred_vec=[]

In [24]:
for file in file_names:
    df=pd.read_csv(files_dir+file)
    df_temp=df[41:]
    lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
    up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
    df1=df_temp[df_temp.loss_var>=lw]
    df1=df1[df1.loss_var<=up]
    
    pred_vec.append(df1.y_pred.mean())

In [25]:
est_vec=np.array(pred_vec)

In [26]:
est_vec.mean()

3.1176130213914117

In [27]:
2*est_vec.std()+ est_vec.mean(),-2*est_vec.std()+ est_vec.mean()

(3.1553098438715836, 3.07991619891124)